## Demo of STMatch Python API

In [1]:
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig

### Load network and graph data

In [2]:
network = Network("../data/edges.shp")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

Nodes 17 edges 30


### Create STMATCH model

In [3]:
model = STMATCH(network,graph)

### Define STMATCH configuration

In [4]:
k = 4
gps_error = 0.5
radius = 0.4
vmax = 30
factor = 1.5
stmatch_config = STMATCHConfig(k, radius, gps_error, vmax, factor)

### Run map matching

#### Match a single trajectory in wkt format

In [5]:
wkt = "LINESTRING(0.200812146892656 2.14088983050848,1.44262005649717 2.14879943502825,3.06408898305084 2.16066384180791,3.06408898305084 2.7103813559322,3.70872175141242 2.97930790960452,4.11606638418078 2.62337570621469)"

In [6]:
result = model.match_wkt(wkt,stmatch_config)

In [7]:
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

Matched path:  [8, 11, 13, 18, 20, 24]
Matched edge for each point:  [8, 11, 18, 18, 20, 24]
Matched edge index  [0, 1, 3, 3, 4, 5]
Matched geometry:  LINESTRING(0.20081215 2,1 2,2 2,3 2,3 3,4 3,4 2.6233757)
Matched point  LINESTRING(0.20081215 2,1.4426201 2,3 2.1606638,3 2.7103814,3.7087218 3,4 2.6233757)


#### Match wkt with timestamps

In [8]:
from fmm import Trajectory,wkt2linestring

In [9]:
line = wkt2linestring(wkt)

In [10]:
traj_id = 1
timestamps = []
for i in range(line.get_num_points()):
    timestamps.append(i)

In [11]:
traj = Trajectory(traj_id,line,timestamps)

In [12]:
result = model.match_traj(traj,stmatch_config)

In [13]:
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())

Matched path:  [8, 11, 13, 18, 20, 24]
Matched edge for each point:  [8, 11, 18, 18, 20, 24]
Matched edge index  [0, 1, 3, 3, 4, 5]
Matched geometry:  LINESTRING(0.20081215 2,1 2,2 2,3 2,3 3,4 3,4 2.6233757)


#### Match GPS data stored in a file

In [14]:
from fmm import GPSConfig,ResultConfig

In [15]:
# Define input data configuration
input_config = GPSConfig()
input_config.file = "../data/trips.csv"
input_config.id = "id"

In [16]:
print(input_config.to_string())

gps file : ../data/trips.csv
id column : id
geom column : geom
timestamp column : timestamp
x column : x
y column : y
GPS point : false



In [17]:
# Define output configuration
result_config = ResultConfig()
result_config.file = "../data/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

Result file : ../data/mr.txt
Output fields: opath cpath mgeom 


In [18]:
status = model.match_gps_file(input_config, result_config, stmatch_config)

In [19]:
print(status)

Status: success
Time takes 0.004 seconds
Total points 17 matched 17
Map match speed 4250 points/s 

